In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Thesis/

/content/drive/MyDrive/Thesis


In [ ]:
import numpy as np
import pandas as pd
from pandas import read_excel
from sklearn.model_selection import train_test_split
import seaborn as sns
import re

In [ ]:
# Dataset after manual annotation
df = read_excel("complete data after manual annotation.xlsx")
df

,Sentences,English_Sentences,Normalized Sentences,Topics
0,Ich habe die PowerPoint Präsentation aufmerksa...,I carefully looked through the PowerPoint pres...,powerpoint präsentation aufmerksam durchgescha...,vorlesungsmaterial_powerpointfolie_vorlesungsv...
1,Ich bin sehr zufrieden mit meinem Lernergebnis.,I am very satisfied with my learning result.,zufrieden lernergebnis,zufriedenstellend_zufrieden_lernergebnis_ergebnis
2,Ich habe den Vorgang gut durchstrukturiert und...,I structured the process well and used the tim...,vorgang gut durchstrukturieren zeit effektiv n...,konzentrieren_konzentriert_bearbeitungszeit_au...
3,Ich hatte viele Informationen nicht wie z.B. w...,"I didn't have a lot of information, such as ho...",vieler information alt sus lehrperson machen e...,klassenmanagment_klassenmanagement_unterrichts...
4,"Mir fiel diese Aufgabe leichter, als die, zum ...",This task was easier for me than the one on th...,fallen aufgabe leicht thema einführung pädagog...,pädagogischen_pädagogisch_pädagogische_diagnos...
...,...,...,...,...
19198,Im Mittelpunkt der Interaktion steht hauptsäch...,The main focus of the interaction is the mutua...,mittelpunkt interaktion stehen hauptsächlich w...,_verständlich_reflexion_beispiel_anwenden
19199,Ausgewählte Beispiele für synchrone Kommunikat...,Describe selected examples of synchronous comm...,ausgewählte beispiel synchron kommunikation de...,_verständlich_reflexion_beispiel_anwenden
19200,Beispiel: Spracherkennungssoftware,Example: speech recognition software,beispiel spracherkennungssoftware,informatikunterricht_technologie_informatik_in...
19201,Mögliche Kontexte: Beschreibung unterschiedlic...,Possible contexts: description of different ex...,möglich kontext beschreibung unterschiedlich b...,informatikunterricht_technologie_informatik_in...


In [ ]:
df['Topics'] = df['Topics'].replace( 'konzentrieren_konzentriert_bearbeitungszeit_aufgabenbearbeitung', 'aufgabe_schreiben_musterlösung_thematik')

In [ ]:
df['Topics'] = df['Topics'].replace( 'klassenmanagment_klassenmanagement_unterrichtsstörung_unterrichtsstunde', 'pädagogischen_pädagogisch_pädagogische_diagnostisch')

In [ ]:
counts = df["Topics"].value_counts()
print(counts)

aufgabe_schreiben_musterlösung_thematik                                                4107
_verständlich_reflexion_beispiel_anwenden                                              3334
pädagogischen_pädagogisch_pädagogische_diagnostisch                                    2910
herausforderung_nachdenken_schwierig_jedoch                                            2054
beibringen_unterrichtsstunde_unterrichten_unterricht                                   1958
vorlesungsmaterial_powerpointfolie_vorlesungsvideo_powerpointpräsentation              1536
informatikunterricht_technologie_informatik_interaktiv                                  642
literaturbezug_literaturübersicht_literaturvorschlag_literatur                          538
konfliktanalyse_konflikanalyse_konfliktsituation_konfliktart                            527
zufriedenstellend_zufrieden_lernergebnis_ergebnis                                       334
feedbackdatei_feedback_feedbackpapier_feedbackbogen                             

Bert Model

In [ ]:
# install transformers package on google colab
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
# Assigning ids to topics
df['Topic labels'] = df['Topics'].apply({'aufgabe_schreiben_musterlösung_thematik': 0, 'sprachunterricht_sprachvorlesung_sprachkenntniss_sprachentwicklung' : 1 ,  'konfliktanalyse_konflikanalyse_konfliktsituation_konfliktart': 2 , 'musikunterricht_musikstunde_musikerziehung_musikalisch': 3 , 'zufriedenstellend_zufrieden_lernergebnis_ergebnis': 4, 'mathematikunterricht_mathematikstunde_mathefähigkeit_mathearbeit': 5 , 'vorlesungsmaterial_powerpointfolie_vorlesungsvideo_powerpointpräsentation': 6 , 'informatikunterricht_technologie_informatik_interaktiv': 7 , 'beibringen_unterrichtsstunde_unterrichten_unterricht': 8 , 'feedbackdatei_feedback_feedbackpapier_feedbackbogen': 9 , 'literaturbezug_literaturübersicht_literaturvorschlag_literatur': 10 , '_interventionsmaßnahme_interventionsmaßnaher_interventionsaufgabe_präventionsansatz': 11 , 'wissenschaft_erkunden_fortbildungsveranstaltung_science': 12 , '_portfoliobearbeitung_portfolioaufgabe_vorstrukturierung_portfolioteil': 13 , '_verständlich_reflexion_beispiel_anwenden': 14 , 'herausforderung_nachdenken_schwierig_jedoch': 15 , 'pädagogischen_pädagogisch_pädagogische_diagnostisch': 16}.get)
df

,Sentences,English_Sentences,Normalized Sentences,Topics,Topic labels
0,Ich habe die PowerPoint Präsentation aufmerksa...,I carefully looked through the PowerPoint pres...,powerpoint präsentation aufmerksam durchgescha...,vorlesungsmaterial_powerpointfolie_vorlesungsv...,6
1,Ich bin sehr zufrieden mit meinem Lernergebnis.,I am very satisfied with my learning result.,zufrieden lernergebnis,zufriedenstellend_zufrieden_lernergebnis_ergebnis,4
2,Ich habe den Vorgang gut durchstrukturiert und...,I structured the process well and used the tim...,vorgang gut durchstrukturieren zeit effektiv n...,aufgabe_schreiben_musterlösung_thematik,0
3,Ich hatte viele Informationen nicht wie z.B. w...,"I didn't have a lot of information, such as ho...",vieler information alt sus lehrperson machen e...,pädagogischen_pädagogisch_pädagogische_diagnos...,16
4,"Mir fiel diese Aufgabe leichter, als die, zum ...",This task was easier for me than the one on th...,fallen aufgabe leicht thema einführung pädagog...,pädagogischen_pädagogisch_pädagogische_diagnos...,16
...,...,...,...,...,...
19198,Im Mittelpunkt der Interaktion steht hauptsäch...,The main focus of the interaction is the mutua...,mittelpunkt interaktion stehen hauptsächlich w...,_verständlich_reflexion_beispiel_anwenden,14
19199,Ausgewählte Beispiele für synchrone Kommunikat...,Describe selected examples of synchronous comm...,ausgewählte beispiel synchron kommunikation de...,_verständlich_reflexion_beispiel_anwenden,14
19200,Beispiel: Spracherkennungssoftware,Example: speech recognition software,beispiel spracherkennungssoftware,informatikunterricht_technologie_informatik_in...,7
19201,Mögliche Kontexte: Beschreibung unterschiedlic...,Possible contexts: description of different ex...,möglich kontext beschreibung unterschiedlich b...,informatikunterricht_technologie_informatik_in...,7


In [ ]:
X = df[['Sentences']]
y = df[['Topic labels']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True)

In [ ]:
X_train.shape

(15362, 1)

In [ ]:
#if want to use data augmentation for train set


USE_DATA_AUGMENTATION = True

if USE_DATA_AUGMENTATION:
  df_aug = read_excel("data_aug_using_duplicates_after_MA.xlsx")

  df_aug['Topics'] = df_aug['Topics'].replace( 'konzentrieren_konzentriert_bearbeitungszeit_aufgabenbearbeitung', 'aufgabe_schreiben_musterlösung_thematik')
  df_aug['Topics'] = df_aug['Topics'].replace( 'klassenmanagment_klassenmanagement_unterrichtsstörung_unterrichtsstunde', 'pädagogischen_pädagogisch_pädagogische_diagnostisch')


  df_aug['Topic labels'] = df_aug['Topics'].apply({'aufgabe_schreiben_musterlösung_thematik': 0, 'sprachunterricht_sprachvorlesung_sprachkenntniss_sprachentwicklung' : 1 ,  'konfliktanalyse_konflikanalyse_konfliktsituation_konfliktart': 2 , 'musikunterricht_musikstunde_musikerziehung_musikalisch': 3 , 'zufriedenstellend_zufrieden_lernergebnis_ergebnis': 4, 'mathematikunterricht_mathematikstunde_mathefähigkeit_mathearbeit': 5 , 'vorlesungsmaterial_powerpointfolie_vorlesungsvideo_powerpointpräsentation': 6 , 'informatikunterricht_technologie_informatik_interaktiv': 7 , 'beibringen_unterrichtsstunde_unterrichten_unterricht': 8 , 'feedbackdatei_feedback_feedbackpapier_feedbackbogen': 9 , 'literaturbezug_literaturübersicht_literaturvorschlag_literatur': 10 , '_interventionsmaßnahme_interventionsmaßnaher_interventionsaufgabe_präventionsansatz': 11 , 'wissenschaft_erkunden_fortbildungsveranstaltung_science': 12 , '_portfoliobearbeitung_portfolioaufgabe_vorstrukturierung_portfolioteil': 13 , '_verständlich_reflexion_beispiel_anwenden': 14 , 'herausforderung_nachdenken_schwierig_jedoch': 15 , 'pädagogischen_pädagogisch_pädagogische_diagnostisch': 16}.get)

  X_aug = df_aug[['Sentences']]
  y_aug = df_aug[['Topic labels']]

  X_train_new = pd.concat([X_train, X_aug], axis=0)
  y_train_new = pd.concat([y_train, y_aug], axis=0)

  X_train = X_train_new
  y_train = y_train_new




In [ ]:
y_train.shape

(19115, 1)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
MODEL_NAME = "xlm-roberta-base"       # Other models used "bert-base-multilingual-cased",  "bert-base-german-cased"

In [ ]:
# Split train data further into training and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.2)

In [ ]:
y_train

In [ ]:
X_train = X_train["Sentences"].tolist()
X_test = X_test["Sentences"].tolist()
X_val = X_val["Sentences"].tolist()

Y_train = y_train["Topic labels"].tolist()
Y_test = y_test["Topic labels"].tolist()
Y_val = y_val["Topic labels"].tolist()

In [ ]:
# pass X's into tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# turn encodings and labels into dataset objects
import numpy as np
import torch


class ReflectivenessDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx])

        return item




In [ ]:
label2id = {'aufgabe_schreiben_musterlösung_thematik': 0, 'sprachunterricht_sprachvorlesung_sprachkenntniss_sprachentwicklung' : 1 ,  'konfliktanalyse_konflikanalyse_konfliktsituation_konfliktart': 2 , 'musikunterricht_musikstunde_musikerziehung_musikalisch': 3 , 'zufriedenstellend_zufrieden_lernergebnis_ergebnis': 4, 'mathematikunterricht_mathematikstunde_mathefähigkeit_mathearbeit': 5 , 'vorlesungsmaterial_powerpointfolie_vorlesungsvideo_powerpointpräsentation': 6 , 'informatikunterricht_technologie_informatik_interaktiv': 7 , 'beibringen_unterrichtsstunde_unterrichten_unterricht': 8 , 'feedbackdatei_feedback_feedbackpapier_feedbackbogen': 9 , 'literaturbezug_literaturübersicht_literaturvorschlag_literatur': 10 , '_interventionsmaßnahme_interventionsmaßnaher_interventionsaufgabe_präventionsansatz': 11 , 'wissenschaft_erkunden_fortbildungsveranstaltung_science': 12 , '_portfoliobearbeitung_portfolioaufgabe_vorstrukturierung_portfolioteil': 13 , '_verständlich_reflexion_beispiel_anwenden': 14 , 'herausforderung_nachdenken_schwierig_jedoch': 15 , 'pädagogischen_pädagogisch_pädagogische_diagnostisch': 16}
id2label = {0: 'aufgabe_schreiben_musterlösung_thematik', 1: 'sprachunterricht_sprachvorlesung_sprachkenntniss_sprachentwicklung' , 2: 'konfliktanalyse_konflikanalyse_konfliktsituation_konfliktart', 3: 'musikunterricht_musikstunde_musikerziehung_musikalisch',  4: 'zufriedenstellend_zufrieden_lernergebnis_ergebnis', 5: 'mathematikunterricht_mathematikstunde_mathefähigkeit_mathearbeit', 6: 'vorlesungsmaterial_powerpointfolie_vorlesungsvideo_powerpointpräsentation', 7: 'informatikunterricht_technologie_informatik_interaktiv', 8: 'beibringen_unterrichtsstunde_unterrichten_unterricht', 9: 'feedbackdatei_feedback_feedbackpapier_feedbackbogen', 10: 'literaturbezug_literaturübersicht_literaturvorschlag_literatur', 11: '_interventionsmaßnahme_interventionsmaßnaher_interventionsaufgabe_präventionsansatz', 12: 'wissenschaft_erkunden_fortbildungsveranstaltung_science', 13: '_portfoliobearbeitung_portfolioaufgabe_vorstrukturierung_portfolioteil', 14: '_verständlich_reflexion_beispiel_anwenden', 15: 'herausforderung_nachdenken_schwierig_jedoch', 16: 'pädagogischen_pädagogisch_pädagogische_diagnostisch'}

In [ ]:
problem_type = "single_label_classification"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 17, problem_type=problem_type, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             f1_score, cohen_kappa_score,
                             confusion_matrix, classification_report)

In [ ]:
def single_label_postprocess(pred):
    pred_proba = softmax(pred, axis=1)  # logits to probabilities
    pred = np.argmax(pred, axis=1)  # logits to one-hot

    return pred_proba, pred

def compute_metrics(p, enable_argmax=False):
    pred, labels = p  # logits and corresponding ground truth
    ret_dict = dict()


    pred_proba, pred = single_label_postprocess(pred)

    ret_dict["Accuracy"] = accuracy_score(y_true=labels, y_pred=pred)
    ret_dict["Recall"] = recall_score(y_true=labels, y_pred=pred, average="weighted",  zero_division=0)
    ret_dict["Precision"] = precision_score(y_true=labels, y_pred=pred, average="weighted", zero_division=0)
    ret_dict["F1 Weighted"] = f1_score(y_true=labels, y_pred=pred, average="weighted", zero_division=0)
    ret_dict["F1 Macro"] = f1_score(y_true=labels, y_pred=pred, average="macro", zero_division=0)
    ret_dict["Cohen's Kappa"] = cohen_kappa_score(y1=labels, y2=pred)

    #print(pred)

    return ret_dict

In [ ]:
import transformers
from torch import nn
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments
from scipy.special import softmax, expit


BATCH_SIZE = 32
NUM_EPOCHS = 25
LEARNING_RATE = 5e-5
EARLY_STOPPING_PATIENCE = 4
USE_EARLY_STOPPING = True


best_model_metric = "F1 Weighted"
greater_is_better = True


training_args = TrainingArguments(
    optim="adamw_torch", # optimizer
    learning_rate=LEARNING_RATE,
    warmup_steps=1000,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir=f'./logs-{MODEL_NAME}',       # directory for storing logs
    num_train_epochs=NUM_EPOCHS,       # total number of training epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE//2, # batch size for evaluation
    output_dir=f'./results-{MODEL_NAME}',     # output directory
    logging_steps=800,                  # interval of logging
    evaluation_strategy='steps',       # what to count for evaluation interval
    eval_steps=800,                     # interval of evaluation
    save_strategy='steps',             # what to count for saving interval
    save_steps=800,                     # interval of checkpoint saving
    save_total_limit=2,                # how many checkpoints to save (best+last)
    greater_is_better=greater_is_better,      # if higher values for metric = better
    metric_for_best_model = best_model_metric,
    load_best_model_at_end=True        # whether to load best model at end
)


callbacks = [EarlyStoppingCallback(early_stopping_patience=EARLY_STOPPING_PATIENCE)] if USE_EARLY_STOPPING else []


#trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset=train_dataset,
#    eval_dataset=val_dataset,
#    compute_metrics=compute_metrics,
#    callbacks=callbacks

#)



6 Epochs

In [ ]:
X_encodings_train = tokenizer(X_train, truncation=True, padding=True)
X_encodings_test = tokenizer(X_test, truncation=True, padding=True)
X_encodings_val = tokenizer(X_val, truncation=True, padding=True)

train_dataset = ReflectivenessDataset(X_encodings_train, Y_train)
test_dataset = ReflectivenessDataset(X_encodings_test, Y_test)
val_dataset = ReflectivenessDataset(X_encodings_val, Y_val)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=callbacks

)




trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1 weighted,F1 macro,Cohen's kappa
800,1.745200,1.065875,0.631510,0.631510,0.646234,0.610439,0.655314,0.576218
1600,0.889500,0.887892,0.695964,0.695964,0.727031,0.684309,0.730192,0.652663
2400,0.644800,0.829518,0.737630,0.737630,0.738636,0.734272,0.767011,0.698764
3200,0.422600,0.991534,0.721680,0.721680,0.723446,0.716839,0.759091,0.682092
4000,0.293900,1.003852,0.735026,0.735026,0.743166,0.734379,0.787036,0.697545
4800,0.217300,1.114182,0.727539,0.727539,0.733099,0.725739,0.780578,0.688651
5600,0.135400,1.409743,0.732747,0.732747,0.744640,0.732346,0.776376,0.694738
6400,0.110500,1.526755,0.736003,0.736003,0.735828,0.729813,0.784360,0.697382
7200,0.084400,1.724565,0.737305,0.737305,0.737507,0.733983,0.781523,0.699764
8000,0.067400,1.820405,0.737956,0.737956,0.741122,0.736120,0.781861,0.700258


TrainOutput(global_step=14950, training_loss=0.25592721545975344, metrics={'train_runtime': 5326.8233, 'train_samples_per_second': 89.711, 'train_steps_per_second': 2.807, 'total_flos': 4.691106582400877e+16, 'train_loss': 0.25592721545975344, 'epoch': 25.0})

In [ ]:
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             f1_score, cohen_kappa_score,
                             confusion_matrix, classification_report)
preds, labels, metrics = trainer.predict(test_dataset)

acc = accuracy_score(y_true=labels, y_pred=np.argmax(preds, axis=1))
print(acc)

0.7633289986996099


In [ ]:
cmatrix = confusion_matrix(y_true=labels, y_pred=np.argmax(preds, axis=1))
print(cmatrix)
print(classification_report(y_true=labels, y_pred=np.argmax(preds, axis=1), zero_division = 0))

[[120   0   0   0   2   0   4   0   1   2   0   0   0   0  14   6   4]
 [  0  11   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  3   0  12   0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   9   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1   0   0   0  11   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   3   0   0   1   0   0   0   0   0   0   0   0]
 [  2   0   0   0   1   0  41   0   2   0   0   0   0   0   3   2   4]
 [  0   0   0   0   0   0   0  29   0   0   0   0   0   0   0   0   0]
 [  2   2   0   0   0   0   2   2  63   0   0   0   0   0   6   4   5]
 [  0   0   0   0   0   0   0   0   0  13   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  21   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0  17   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   2   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   4   0   0   0]
 [ 14 

In [ ]:
import csv

# save metrics
with open('bert_model_after_MA_roberta_after_aug_latest.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["metric", "value"])
    for key, value in metrics.items():
       writer.writerow([key, value])

# save model
model.save_pretrained("/content/drive/MyDrive/Thesis/bert_model_roberta_aug_latest")

Trying Cross Validation with differnt parameters and models

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


X = df[['Sentences']]
y = df[['Topic labels']]
count = 1


pred_acc = []
pred_f1_w = []
pred_f1_mac = []

kf = KFold(n_splits=3, random_state=99, shuffle=True)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.1, random_state=40)

    # Train
    X_train = X_train["Sentences"].tolist()
    X_test = X_test["Sentences"].tolist()
    X_val = X_val["Sentences"].tolist()
    #Test
    Y_train = y_train["Topic labels"].tolist()
    Y_test = y_test["Topic labels"].tolist()
    Y_val = y_val["Topic labels"].tolist()

# Also use varying learning rates and models to check
    if count==1:
      BATCH_SIZE = 8

    elif count==2:
      BATCH_SIZE = 16

    elif count==3:
      BATCH_SIZE = 32



    # pass X's into tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    X_encodings_train = tokenizer(X_train, truncation=True, padding=True)
    X_encodings_test = tokenizer(X_test, truncation=True, padding=True)
    X_encodings_val = tokenizer(X_val, truncation=True, padding=True)

    train_dataset = ReflectivenessDataset(X_encodings_train, Y_train)
    test_dataset = ReflectivenessDataset(X_encodings_test, Y_test)
    val_dataset = ReflectivenessDataset(X_encodings_val, Y_val)







    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 17, problem_type=problem_type, label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True)


    training_args = TrainingArguments(
    optim="adamw_torch", # optimizer
    learning_rate=LEARNING_RATE,
    warmup_steps=1000,                  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # strength of weight decay
    logging_dir=f'./logs-{MODEL_NAME}',       # directory for storing logs
    num_train_epochs=NUM_EPOCHS,       # total number of training epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE//2, # batch size for evaluation
    output_dir=f'./results-{MODEL_NAME}',     # output directory
    logging_steps=500,                  # interval of logging
    evaluation_strategy='steps',       # what to count for evaluation interval
    eval_steps=500,                     # interval of evaluation
    save_strategy='steps',             # what to count for saving interval
    save_steps=500,                     # interval of checkpoint saving
    save_total_limit=2,                # how many checkpoints to save (best+last)
    greater_is_better=greater_is_better,      # if higher values for metric = better
    metric_for_best_model = best_model_metric,
    load_best_model_at_end=True        # whether to load best model at end
)



    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    )


    trainer.train()

    count = count + 1


    preds, labels, metrics = trainer.predict(test_dataset)

    acc = accuracy_score(y_true=labels, y_pred=np.argmax(preds, axis=1))
    pred_acc.append(acc)

    print("Testing Accuracy: ", acc)

    f1_we = f1_score(y_true=labels, y_pred=np.argmax(preds, axis=1), average="weighted", zero_division=0)
    f1_mac = f1_score(y_true=labels, y_pred=np.argmax(preds, axis=1), average="macro", zero_division=0)
    pred_f1_w.append(f1_we)
    pred_f1_mac.append(f1_mac)
    print("Testing F1_Weighted: ", f1_we)
    print("Testing F1_Weighted: ", f1_mac)


